In [1]:
### Dataloader ###
# From PDF to CSV
# Here, the data is extracted from the PDF files and merged with the Schultz main data before being stored as a CSV file. 
# The preparation of the main data is done in a separate script.
# This process is done separately for each commune before being merged together.
# Note: It's not possible to run the scripts since the PDF files are not available.

In [1]:
# Import libraries
import pandas as pd          # To handle and manipulate dataframes
import os                    # To interact with the operating system
import pdfplumber            # To extract text from PDF files
import re                    # To use regular expressions
from tqdm import tqdm        # To display progress bars for loops


In [ ]:
# Main Data Loader #

# The folder containing the PDF files
pdf_folder = 'C:\\Users....'

# List to store data extracted from the PDF files
data = []

# Retrieve a list of all files in pdf_folder, including subdirectories
all_files = [os.path.join(root, filename) for root, dirs, files in os.walk(pdf_folder) for filename in files]

# Use tqdm to display a progress bar
for pdf_path in tqdm(all_files, desc="Processing PDF files"):
    try:
        # Open each file as a PDF
        with pdfplumber.open(pdf_path) as pdf:
            # Loop through all pages in the PDF file
            for page in pdf.pages:
                # Extract text from the current page
                text = page.extract_text()
                
                # Add the data to the list
                data.append({'PDF Filename': os.path.basename(pdf_path), 'Text': text})
    except Exception as e:
        print(f"Error processing file {os.path.basename(pdf_path)}: {e}")

# The 'data' list now contains information from all successfully processed PDF files, except those that caused errors


In [ ]:
# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data)

# Remove duplicate rows based on the 'PDF Filename' column
df = df.drop_duplicates(subset=['PDF Filename'])

# Retrieve the text content of the first two unique PDF files
file1_text_1 = df.iloc[0]['Text']
file1_text_2 = df.iloc[1]['Text']

# Identify differences between the two text strings by comparing characters at each index
difference = [i for i in range(min(len(file1_text_1), len(file1_text_2))) if file1_text_1[i] != file1_text_2[i]]


In [2]:
# Selection of variables from all the information forms

In [ ]:
# Social Security Number - There are differences based on A, B, and C
df['CPR-nummer'] = df['Tekst'].str.extract('CPR-nr\. (\d{10})')

# "Hvad fejler du?"
df['Hvad fejler du'] = df['Tekst'].str.extract(r'Hvad fejler du\?\s*([^.\n]+)')

# "Uddyb kort og præcist..."
pattern = r'Uddyb kort og præcist, hvilken sygdom der er årsag til dit fravær:(.*?)\n'

# Illness 
df['Sygdoms årsag'] = df['Tekst'].str.extract(pattern)

# Arbejdsskadestyrelsen
pattern = r'Er arbejdsskaden anmeldt til Arbejdsskadestyrelsen\? (\w+)'

# regex pattern to extract if the injury has been reported
df['Arbejdsskaden anmeldt'] = df['Tekst'].str.extract(pattern)

# Extract previous illness data using different patterns
df['Syg1'] = df['Tekst'].str.extract(r'Har du tidligere været sygemeldt af[\s\S]*?(Ja|Nej|ja|nej)[\s\S]*?samme årsag\?')
df['Syg2'] = df['Tekst'].str.extract(r'Har du tidligere været sygemeldt af samme årsag\?\s*(Ja|Nej|ja|nej)')

# A new regex pattern to handle line breaks in the question
df['Syg3'] = df['Tekst'].str.extract(r'Har du tidligere været[\s\S]*?(Ja|Nej|ja|nej)[\s\S]*?samme årsag\?')

# Combine the results from the different patterns into one column
df['Syg af samme årsag'] = df['Syg1'].combine_first(df['Syg2']).combine_first(df['Syg3'])

# Remove temporary columns used for intermediate results
df.drop(['Syg1', 'Syg2', 'Syg3'], axis=1, inplace=True)


In [ ]:
# life-threatening
df['Livstruende sygdom'] = df['Tekst'].str.extract(r'Er din sygdom livstruende\?\s*(\w+)')

# Work-related injury
df['Arbejdsskade'] = df['Tekst'].str.extract(r'(?:Skyldes\s+dit\s+fravær\s+en\s+arbejdsskade\?|Forventer\s+du\s+at\s+vende\s+tilbage)\s*[\s\S]*?\s*(ja|nej|Ja|Nej)')

# Arbejdsskadestyrelsen
pattern = r'Er arbejdsskaden anmeldt til Arbejdsskadestyrelsen\? (\w+)'
df['Arbejdsskaden anmeldt'] = df['Tekst'].str.extract(pattern)

# Accident/injury/assault
df['Ulykke/tilskadekomst/overfald'] = df['Tekst'].str.extract(r'Skyldes\s+dit\s+fravær[\s\S]+?(?:ulykke|tilskadekomst|overfald)[\s\S]+?\?\s*\(\s*personskade\s*\)[\s\S]+?(Ja|Nej|ja|nej)')

# Job position
df['Stilling'] = df['Tekst'].str.extract(r'Din stilling:\s*(.+?)(?=\n|[A-Z]|$)')

# Education information
pattern = r'Din uddannelse: ([^\n]+)'
df[['Uddannelse']] = df['Tekst'].str.extract(pattern)

# Axpected recovery date
pattern = r'(\d{1,2}\.\s+\w+\s+\d{4})'
df['Forventet raskmeldingsdato'] = df['Tekst'].str.extract(pattern)
df['Forventet raskmeldingsdato'] = df['Forventet raskmeldingsdato'].str.replace('\n', ' ').str.strip()

# Extract if the employee expects to return to their workplace
def extract_answer(text, question_start):
    lines = text.split('\n')
    for i, line in enumerate(lines):
        if question_start in line:
            # Answer on the same line
            if "Ja" in line or "Nej" in line:
                return re.search(r'(Ja|Nej)', line).group(1)
            # Answer on the next line
            elif i + 1 < len(lines) and (lines[i + 1].strip() == "Ja" or lines[i + 1].strip() == "Nej"):
                return lines[i + 1].strip()
    return None

df['Tilbage til arbejdspladsen'] = df['Tekst'].apply(lambda x: extract_answer(x, "Forventer du at vende tilbage"))

# Undergoing treatment for their illness
df['Behandling'] = df['Tekst'].str.extract(r'Er du i behandling for din sygdom\?\s*(\w+)')

# Awaiting surgery
df['Afventer operation'] = df['Tekst'].str.extract(r'Afventer du en operation\?\s*(\w+)')

# Person's work before they were signed off
def find_answers(text):
    answer_matches = re.findall(r'Har du, eller er du ved at få en.*?(ja|nej)\b', text, re.IGNORECASE | re.DOTALL)
    if answer_matches:
        return [match.lower() for match in answer_matches]  # Returns "ja" or "nej"
    return []

# Apply the function to find the answers and remove empty lists
df['Påvirker sygdom arbejdet'] = df['Tekst'].apply(lambda x: find_answers(x))
df['Påvirker sygdom arbejdet'] = df['Påvirker sygdom arbejdet'].apply(lambda x: x[0] if x else None)

# Amployer's response
df['Din arbejdsgivers svar:'] = df['Tekst'].str.extract(r'Din arbejdsgivers svar:\s*(.+)')

# Citizen's response
df['Dit svar:'] = df['Tekst'].str.extract(r'Dit svar:\s*(.+)')


In [ ]:
# Illness affected your work? Second responses from the employer and citizen

def find_second_occurrence(text, pattern):
    matches = re.finditer(pattern, text)
    for i, match in enumerate(matches):
        if i == 1:  # Second occurrence (indexing starts at 0)
            return match.group(1)
    return None  # Return None if there is no second occurrence

# Second occurrence of the employer's response
pattern = r'Din arbejdsgivers svar:\s*(.+)'
df['Din arbejdsgivers andet svar:'] = df['Tekst'].apply(lambda x: find_second_occurrence(x, pattern))

# Second occurrence of the citizen's response
pattern = r'Dit svar:\s*(.+)'
df['Dit andet svar:'] = df['Tekst'].apply(lambda x: find_second_occurrence(x, pattern))

# Extract third responses from the employer and citizen

def find_third_occurrence(text, pattern):
    matches = re.finditer(pattern, text)
    for i, match in enumerate(matches):
        if i == 2:  # Third occurrence (indexing starts at 0)
            return match.group(1)
    return None  # Return None if there is no third occurrence

# Third occurrence of the employer's response
pattern = r'Din arbejdsgivers svar:\s*(.+)'
df['Din arbejdsgivers tredje svar:'] = df['Tekst'].apply(lambda x: find_third_occurrence(x, pattern))

# Third occurrence of the citizen's response
pattern = r'Dit svar:\s*(.+)'
df['Dit tredje svar:'] = df['Tekst'].apply(lambda x: find_third_occurrence(x, pattern))

# Plan for returning to work? 

def find_answers(text):
    # Find "yes" or "no" answers related to the question
    answer_matches = re.findall(r'Har du og din arbejdsgiver.*?(ja|nej)\b', text, re.IGNORECASE | re.DOTALL)
    if answer_matches:
        return [match.lower() for match in answer_matches]  # Return answers in lowercase
    return []

# Function to the DataFrame and remove empty lists
df['Tilbagevenden til arbejdet'] = df['Tekst'].apply(lambda x: find_answers(x))
df['Tilbagevenden til arbejdet'] = df['Tilbagevenden til arbejdet'].apply(lambda x: x[0] if x else None)

# Do you have, or are you in the process of obtaining a medical certificate for your employer?

def find_answers(text):
    # Find "yes" or "no" answers related to the question
    answer_matches = re.findall(r'Har du, eller er du ved at få en.*?(ja|nej)\b', text, re.IGNORECASE | re.DOTALL)
    if answer_matches:
        return [match.lower() for match in answer_matches]  # Return answers in lowercase
    return []

# Function to the DataFrame and remove empty lists
df['Lægeerklæring'] = df['Tekst'].apply(lambda x: find_answers(x))
df['Lægeerklæring'] = df['Lægeerklæring'].apply(lambda x: x[0] if x else None)


In [3]:
# Minor data cleaning

In [ ]:
# Remove Unwanted Columns
df = df.drop(['PDF Filnavn', 'Tekst'], axis=1)

In [ ]:
# Remove all rows where the column has NaN values
df = df.dropna(subset=['Arbejdsskade', 'Arbejdsskade', 'Ulykke/tilskadekomst/overfald', 'Tilbagevenden til arbejdet', 
             'Påvirker sygdom arbejdet', 'Tilbage til arbejdspladsen', 'Lægeerklæring'])

In [ ]:
# Remove duplicates, keeping only the first occurrence of each CPR number
df = df.drop_duplicates(subset='CPR-nummer', keep='first')

In [ ]:
# Update 'Arbejdsskaden anmeldt' to 'Nej' if 'Arbejdsskade' is 'Nej' and 'Arbejdsskaden anmeldt' is NaN
df.loc[(df['Arbejdsskade'] == 'Nej') & (df['Arbejdsskaden anmeldt'].isna()), 'Arbejdsskaden anmeldt'] = 'Nej'

In [4]:
# Merge the scraped data with the main data.

In [ ]:
# Load the CSV file
csv_df = pd.read_csv('C:\\Users\\...', dtype={'CPR-nummer': str})

In [ ]:
# Perform a right join on 'CPR-nummer' to merge 
merged_df = df.merge(csv_df, on='CPR-nummer', how='right')

In [ ]:
# Save the merged DataFrame to a CSV file named 'commune01.csv', excluding the index
merged_df.to_csv('commune01.csv', index=False)